In [23]:
import pandas as pd
customers = pd.read_csv("../data/external/customers.csv", dtype=str)

customers.head()

,shopUserId,invoiceFirstName,invoiceLastName,invoiceSSN,invoiceZip,invoiceCity,invoiceCountryId,invoiceEmail
0,912267,Ann-Mari,Granlund,194203307329,79023,Svärdsjö,205,8135-1759755027.fake@email.com
1,912257,Marita,Müntzing,194705045229,53143,Lidköping,205,8693-1759754685.fake@email.com
2,418342,Ruth,Iversen,2607521762,6541,Bevtoft,58,ruth@justiversen.dk
3,912250,Ulla,Larsson,193703052401,54237,Mariestad,205,annika@kontrastdesign.es
4,912245,Mona,Jensen,NaN,3250,Gilleleje,58,6396-1759753798.fake@email.com


In [24]:

customers.isnull().mean()

shopUserId          0.000000
invoiceFirstName    0.000000
invoiceLastName     0.000000
invoiceSSN          0.361263
invoiceZip          0.000000
invoiceCity         0.000000
invoiceCountryId    0.000000
invoiceEmail        0.000000
dtype: float64

In [25]:
#count all rows
print(len(customers))

86408


In [26]:
# Output a sample of 10 invoiceSSN values and their counts
customers['invoiceSSN'].value_counts(dropna=False).head(10)


invoiceSSN
NaN             31216
temp             2357
195903311149       34
240343-4727        25
21034730684        21
196112062705       19
030538-006Y        18
194303291688       18
250962-1740        18
180239-063A        17
Name: count, dtype: int64

In [27]:
# Display all rows where invoiceSSN is '141052-138Y'
from IPython.display import display
display(customers[customers['invoiceSSN'] == '15044437403'])


,shopUserId,invoiceFirstName,invoiceLastName,invoiceSSN,invoiceZip,invoiceCity,invoiceCountryId,invoiceEmail
18824,812880,Else Marie,Monsen,15044437403,2760,Brandbu,160,4037-1752651873.fake@email.com
26170,758448,Else Marie,Monsen,15044437403,2760,Brandbu,160,3695-1747732356.fake@email.com
29170,742757,Else Marie,Monsen,15044437403,2760,Brandbu,160,8396-1746691669.fake@email.com
30067,739610,Else Marie,Monsen,15044437403,2760,Brandbu,160,1833-1746521204.fake@email.com
46435,630646,Else Marie,Monsen,15044437403,2760,Brandbu,160,3621-1738928969.fake@email.com
61081,458194,Else Marie,Monsen,15044437403,2760,Brandbu,160,2032-1730454239.fake@email.com
66270,410369,Else Marie,Monsen,15044437403,2760,Brandbu,160,2828-1727951617.fake@email.com
77433,333261,Else Marie,Monsen,15044437403,2760,Brandbu,160,6541-1723106503.fake@email.com
79712,308458,Else Marie,Monsen,15044437403,2760,Brandbu,160,5947-1721283346.fake@email.com
80702,296247,Else Marie,Monsen,15044437403,2760,Brandbu,160,6459-1720169085.fake@email.com


Clean shopUserId name lastname zip

In [28]:
import pandas as pd, numpy as np

# Use pandas' dedicated string dtype for better string handling
S = pd.StringDtype()

def clean(s: pd.Series) -> pd.Series:
    """
    Clean a pandas Series of strings by:
    - Converting to pandas StringDtype for consistent string operations
    - Normalizing unicode characters to NFKC form (e.g., full-width to half-width)
    - Replacing all runs of whitespace with a single space
    - Stripping leading/trailing whitespace
    - Masking (setting to <NA>) any value that matches common null/empty patterns
      such as '', 'nan', 'null', 'none', 'n/a', 'n.a.', '-', '.', or '0' (case-insensitive)
    """
    return (
        s.astype(S)
         .str.normalize("NFKC")
         .str.replace(r"\s+", " ", regex=True)
         .str.strip()
         .mask(lambda x: x.str.fullmatch(
             r"(?i)^(|nan|null|none|n/?a|n\.a\.|-|\.|0)$", na=False
         ))
    )

# Read customer and transaction data as strings, disabling default NA parsing
customers = pd.read_csv(
    "../data/external/customers.csv",
    dtype=str,
    keep_default_na=False,
    na_values=[]
)
tx = pd.read_csv(
    "../data/external/transactions.csv",
    dtype=str,
    keep_default_na=False,
    na_values=[]
)

# Clean selected columns in the customers DataFrame
for col in ["shopUserId", "invoiceFirstName", "invoiceLastName", "invoiceZip"]:
    if col in customers:
        customers[col] = clean(customers[col])

# Clean the shopUserId column in the transactions DataFrame
tx["shopUserId"] = clean(tx["shopUserId"])


In [29]:
print(len(tx))

312079


## Normalize city names

In [30]:
import re

COUNTRY_TAIL_RE = re.compile(r'(?:,\s*)?(Denmark|Danmark|Sweden|Sverige|Norway|Norge|Finland|Suomi)\s*$', re.IGNORECASE)
LEADING_POSTAL_RE = re.compile(r'^(?:[A-Z]{1,3}[-\s])?\d{2,3}\s?\d{2,3}\s+|^(?:[A-Z]{1,3}[-\s])?\d{3,6}\s+', re.IGNORECASE)
DIGITS_ANYWHERE_RE = re.compile(r'\d+')

def sentense_case(s):
    if not s:
        return s
    s = s.strip()
    if not s:
        return s
    return s[0].upper() + s[1:].lower() if len(s) > 1 else s.upper()

def normalize_city(x):
    if x is None or pd.isna(x):
        return "Unknown"
    s = str(x)
    s = re.sub(r"\s+", " ", s).strip(" ,")
    if s == "":
        return "Unknown"
    s = LEADING_POSTAL_RE.sub("", s).strip(" ,")
    s = COUNTRY_TAIL_RE.sub("", s).strip(" ,")
    s = s.strip()
    # Remove all digits anywhere in the string (e.g., "Tyresö13540 Tyresö" -> "Tyresö Tyresö")
    s = DIGITS_ANYWHERE_RE.sub("", s)
    s = s.strip()
    # Handle cases like "Raadal- Bergen" or "Raadal - Bergen" -> "Bergen"
    if "-" in s:
        parts = [p.strip() for p in s.split("-") if p.strip()]
        if len(parts) > 1:
            # Use the last part as the actual city
            s = parts[-1]
    # Remove trailing one-letter words (e.g., "Nykøbing m" -> "Nykøbing")
    s = re.sub(r'\b\w\b$', '', s).strip(" ,")
    # Remove any remaining one-letter words anywhere in the string
    s = ' '.join([word for word in s.split() if len(word) > 1])
    # Convert to sentense case
    s = sentense_case(s)
    return "Unknown" if s in {"", "<NA>", "nan", "NaN"} else s

customers['invoiceCity'] = customers['invoiceCity'].apply(normalize_city)


In [31]:
# Show all unique values of invoiceCity as strings, including NaN, and print them fully (not truncated), along with their counts
pd.set_option('display.max_rows', None)
city_counts = customers['invoiceCity'].astype(str).value_counts(dropna=False)
for city, count in city_counts.items():
    print(f"{city}: {count}")
pd.reset_option('display.max_rows')


Oslo: 2349
Helsinki: 997
Stockholm: 796
Göteborg: 766
Uppsala: 588
Malmö: 486
Västerås: 455
Trondheim: 422
Espoo: 413
Örebro: 396
Drammen: 357
København: 354
Linköping: 350
Turku: 332
Vantaa: 326
Tampere: 298
Lund: 290
Eskilstuna: 281
Umeå: 279
Norrköping: 275
Karlstad: 256
Kristiansand: 254
Moss: 251
Järfälla: 248
Gävle: 247
Bromma: 246
Helsingborg: 241
Skien: 239
Växjö: 239
Oulu: 238
Täby: 230
Sandefjord: 225
Luleå: 224
Norrtälje: 223
Falun: 216
Jönköping: 214
Nyköping: 208
Hägersten: 208
Bergen: 205
Frederiksberg: 200
Lidingö: 198
Sundsvall: 193
Stavanger: 193
Solna: 192
Södertälje: 191
Sollentuna: 185
Kalmar: 183
Askim: 176
Trollhättan: 175
Västra frölunda: 173
Borås: 170
Tromsø: 167
Lahti: 163
Bodø: 162
Halmstad: 160
Östersund: 160
Borlänge: 158
Tyresö: 158
Hämeenlinna: 157
Uddevalla: 153
Sandnes: 149
Jyväskylä: 149
Huddinge: 148
Horten: 146
Hässelby: 145
Lillehammer: 142
Porsgrunn: 141
Åkersberga: 138
Alingsås: 137
Roskilde: 137
Larvik: 136
Sundbyberg: 136
Skövde: 135
Enköping: 1

## Different shopUserId are assigned to the same invoiceSSN -remap in both customers.csv and transactions.csv

In [32]:
# Remap shopUserId so that customers with the same (first name, last name, zip)
# get a canonical shopUserId (first seen by original row order).

# Only use rows where all three fields are present
mask = customers[["invoiceFirstName", "invoiceLastName", "invoiceZip"]].notna().all(axis=1)
base = customers.loc[mask, ["shopUserId", "invoiceFirstName", "invoiceLastName", "invoiceZip"]].copy()

# Key per person
base["key"] = base["invoiceFirstName"] + "||" + base["invoiceLastName"] + "||" + base["invoiceZip"]

# For each key: canonical = first ID by original order; all_ids = unique IDs
canon = (
    base.sort_index()  # preserves original file order
        .groupby("key", as_index=False)["shopUserId"]
        .agg(canonical="first", all_ids=lambda s: list(pd.unique(s)))
)

# Explode to map every observed id -> canonical
remap = (
    canon.explode("all_ids")
         .rename(columns={"all_ids": "shopUserId", "canonical": "canonical_shopUserId"})
         [["shopUserId", "canonical_shopUserId"]]
         .dropna()
         .drop_duplicates()
)

# Ensure canonical IDs map to themselves
remap = pd.concat(
    [
        remap,
        remap[["canonical_shopUserId"]]
            .rename(columns={"canonical_shopUserId": "shopUserId"})
            .assign(canonical_shopUserId=lambda d: d["shopUserId"])
    ],
    ignore_index=True
).drop_duplicates()

# Build mapping dict and apply
m = pd.Series(remap["canonical_shopUserId"].values, index=remap["shopUserId"].values).to_dict()
customers["shopUserId"] = customers["shopUserId"].map(m).fillna(customers["shopUserId"])
tx["shopUserId"] = tx["shopUserId"].map(m).fillna(tx["shopUserId"])

# (optional sanity print)
print(
    "AFTER remap — tx IDs not in customers:",
    len(
        pd.Index(tx["shopUserId"].dropna().unique())
        .difference(pd.Index(customers["shopUserId"].dropna().unique()))
    )
)

AFTER remap — tx IDs not in customers: 0


In [33]:
# The following code is used to assign a city to each transaction based on the canonical customer information.
# Since a single shopUserId might have multiple associated cities (due to data inconsistencies or changes over time),
# we need a way to pick a representative city for each shopUserId. The function below does this by choosing the most
# frequent (mode) city for each shopUserId. If there is no mode (e.g., all values are NaN or equally frequent), it falls
# back to the first non-null value, or pd.NA if none exist.

def mode_or_first(s: pd.Series):
    # Get the mode (most common value) of the series, ignoring NaNs
    m = s.mode(dropna=True)
    if not m.empty:
        # If there is a mode, return the first one (in case of ties)
        return m.iat[0]
    else:
        # If no mode, return the first non-null value if available, else pd.NA
        non_null = s.dropna()
        return non_null.iat[0] if non_null.size else pd.NA

# For each canonical shopUserId, determine their representative city using the above function
cust_city = (
    customers.groupby("shopUserId")["invoiceCity"]
    .agg(mode_or_first)
    .astype(S)  # S is likely a string dtype, ensuring consistent type
)

# Map the representative city from customers to each transaction by shopUserId.
# If a transaction's shopUserId is not found in the mapping, assign "Unknown".
tx["invoiceCity"] = tx["shopUserId"].map(cust_city).fillna("Unknown")

# Print the percentage of transactions with an unknown city
print("Unknown city rate:", round(100 * tx["invoiceCity"].eq("Unknown").mean(), 2), "%")
# Print the number of unique cities present in the transactions
print("Unique cities in tx:", tx["invoiceCity"].nunique(dropna=True))
# Show a sample of shopUserIds for which the city could not be determined
print(tx.loc[tx["invoiceCity"].eq("Unknown"), "shopUserId"].head(10))

# Deduplicate customers by shopUserId, keeping the first occurrence
customers = customers.drop_duplicates(subset=["shopUserId"], keep="first")


tx.to_parquet("../data/processed/transactions_canonical.parquet", index=False)


Unknown city rate: 0.0 %
Unique cities in tx: 4904
133727    669214
Name: shopUserId, dtype: object


In [34]:
print(len(customers))

61840


In [35]:
# Display all rows where invoiceSSN is '141052-138Y'
from IPython.display import display
display(customers[customers['invoiceSSN'] == '15044437403'])


,shopUserId,invoiceFirstName,invoiceLastName,invoiceSSN,invoiceZip,invoiceCity,invoiceCountryId,invoiceEmail
18824,812880,Else Marie,Monsen,15044437403,2760,Brandbu,160,4037-1752651873.fake@email.com


In [36]:
import re
from datetime import date

country_map = {"58": "DK", "160": "NO", "205": "SE", "72": "FI"}

def _safe_date(y, m, d):
    try:
        return date(int(y), int(m), int(d))
    except Exception:
        return None

def _age_from_birthdate(born):
    if not born:
        return None
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

def _parse_birthdate(ssn_str, country):
    if pd.isna(ssn_str):
        return None
    digits = re.sub(r'\D', '', str(ssn_str))

    if country == 'SE':
        if len(digits) >= 12:  # YYYYMMDDxxxx
            return _safe_date(digits[:4], digits[4:6], digits[6:8])
        elif len(digits) >= 10:  # YYMMDDxxxx
            yy, mm, dd = int(digits[:2]), digits[2:4], digits[4:6]
            sep = '-' if '-' in str(ssn_str) else '+' if '+' in str(ssn_str) else None
            this_year = date.today().year
            if sep == '+':  # 100+ years old
                y = 1900 + yy if (1900 + yy) <= this_year - 100 else 1800 + yy
            else:
                y = 1900 + yy if (1900 + yy) > this_year - 100 else 2000 + yy
            return _safe_date(y, mm, dd)
        return None

    if country == 'NO' and len(digits) == 11:
        dd, mm, yy = int(digits[0:2]), int(digits[2:4]), int(digits[4:6])
        individ = int(digits[6:9])
        if dd > 40:  # D-number adjustment
            dd -= 40
        if 0 <= individ <= 499:
            year = 1900 + yy
        elif 500 <= individ <= 749 and 54 <= yy <= 99:
            year = 1800 + yy
        elif 500 <= individ <= 999 and 0 <= yy <= 39:
            year = 2000 + yy
        elif 900 <= individ <= 999 and 40 <= yy <= 99:
            year = 1900 + yy
        else:
            year = (2000 + yy) if yy <= 24 else (1900 + yy)
        return _safe_date(year, mm, dd)

    if country == 'DK' and len(digits) >= 10:
        dd, mm, yy = digits[0:2], digits[2:4], int(digits[4:6])
        year = (2000 + yy) if yy <= 24 else (1900 + yy)
        return _safe_date(year, mm, dd)

    if country == 'FI':
        m = re.match(r'^(\d{2})(\d{2})(\d{2})([-+A])(\d{3})\w?$', str(ssn_str).strip(), re.I)
        if m:
            dd, mm, yy, cent = int(m.group(1)), int(m.group(2)), int(m.group(3)), m.group(4).upper()
            base = {'+': 1800, '-': 1900, 'A': 2000}[cent]
            return _safe_date(base + yy, mm, dd)
        if len(digits) >= 10:  # Old numeric-only fallback
            dd, mm, yy = int(digits[0:2]), int(digits[2:4]), int(digits[4:6])
            year = (2000 + yy) if yy <= 24 else (1900 + yy)
            return _safe_date(year, mm, dd)
        return None

    return None

def get_gender_age_from_ssn(ssn, country_id):
    if pd.isna(ssn):
        return None, None

    ssn_str = str(ssn).strip()
    country_id_str = str(country_id)
    country = country_map.get(country_id_str)
    if not country:
        return None, None

    digits = re.sub(r'\D', '', ssn_str)
    gender_digit = None
    if country == 'SE' and len(digits) >= 10:
        gender_digit = int(digits[-4:][2])
    elif country == 'NO' and len(digits) == 11:
        gender_digit = int(digits[8])
    elif country == 'DK' and len(digits) >= 10:
        gender_digit = int(digits[-1])
    elif country == 'FI':
        m = re.match(r'^\d{6}[-+A]\d{3}\w?$', ssn_str, re.I)
        if m:
            # In Finnish SSN, the gender digit is the 9th character (index 8 in digits)
            gender_digit = int(digits[8])
        elif len(digits) >= 10:
            gender_digit = int(digits[8])

    gender = None
    if gender_digit is not None:
        gender = 'Male' if gender_digit % 2 else 'Female'
    age = _age_from_birthdate(_parse_birthdate(ssn_str, country))
    return gender, age

# Apply to DataFrame
customers[['Gender', 'Age']] = customers.apply(
    lambda r: pd.Series(get_gender_age_from_ssn(r['invoiceSSN'], r['invoiceCountryId'])),
    axis=1
)


In [37]:
# In-place filter: keep rows with Age between lo and hi inclusive, or Age is NA
mask = customers["Age"].isna() | ((customers["Age"] >= 10) & (customers["Age"] <= 105))
customers = customers.loc[mask].reset_index(drop=True)

In [38]:
# Ensure correct mapping and types
country_map = {58: 'Denmark', 205: 'Sweden', 160: 'Norway', 72: 'Finland'}
# invoiceCountryId is object, so map after converting to int where possible
def map_country(val):
    try:
        return country_map.get(int(val), val)
    except (ValueError, TypeError):
        return val

customers['Country'] = customers['invoiceCountryId'].apply(map_country).astype(str)

In [39]:
print(len(customers))

61837


In [40]:
customers.to_parquet('../data/processed/customers_clean.parquet', index=False)

In [41]:
customers.isna().sum()



shopUserId              0
invoiceFirstName        0
invoiceLastName         0
invoiceSSN              0
invoiceZip              0
invoiceCity             0
invoiceCountryId        0
invoiceEmail            0
Gender              24493
Age                 24493
Country                 0
dtype: int64